<a href="https://colab.research.google.com/github/kevinrchilders/computational-number-theory/blob/master/cryptography_chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions

In [ ]:
import numpy as np

In [ ]:
# Fast powering algorithm, gcd algorithms, etc.

def binary(n):
  binary_repn = []
  if n > 1:
    binary_repn = binary(n // 2)
  binary_repn.append(n % 2)
  return binary_repn

def power(g, A, N):
  A = binary(A)
  total=1
  for i in range(len(A)):
    if A[len(A)-i-1]:
      total = (total*g) % N
    g = (g*g) % N
  return total

def gcd(a, b):
  return a if b==0 else gcd(b, a%b)

def extended_gcd(a, b):
  u, g, x, y = 1, a, 0, b
  while y != 0:
    q, t = g // y, g % y
    s = u - q*x
    u, g = x, y
    x, y = s, t
  v = (g - a*u) // b 
  return g, u, v

def order(a, p):
  n = 1
  x = a
  while power(a, n, p) != 1:
    x = (x * a) % p
    n += 1
  return n

def is_primitive(a, p):
  return order(a, p) == p-1

def is_mrprime(n, trials=50):
  for i in range(trials):
    a = int(np.random.rand()*n)
    if is_mrwitness(a, n):
      return False
  return True

def generate_prime(digits, attempts=100):
  N = 2 * 3 * 5 * 7
  for K in range(int(10**(digits)/N), int(10**(digits)/N)+attempts):
    if is_mrprime(N*K + 1):
      return N*K + 1
  print('No primes found. Try more attempts.')
  return None

def is_mrwitness(a, n):
  # If a and n have a common factor, then n is composite
  if gcd(a, n) != 1:
    return True

  # Write n-1 = 2^k*q with q odd
  k=0
  q=n-1
  while q%2 == 0:
    k += 1
    q = q//2
  
  # If a^q == 1 (mod n) then a is not a Miller-Rabin witness for n
  a = power(a, q, n)
  if a == 1:
    return False
  
  # If a^(2^iq) == -1 (mod n) for some i<k then a is not a Miller-Rabin witness for n
  for i in range(k):
    if a == n-1:
      return False
    a = power(a, 2, n)

  return True # Otherwise a is a Miller-Rabin witness for n

def pollard(N, a=2, maxn=1000000):
  for j in np.arange(2, maxn):
    a = power(a, j, N)
    d = gcd(a-1, N)
    if d != 1 and d != N:
      return d
  print('Test failed, try a larger maxn.')
  return None

def find_primitive(p):
  a = 2
  while not is_primitive(a, p):
    a += 1
  return a

# RSA digital signitures

In [ ]:
# An example of Samantha sending a signed document to Victor for verification

p = 5463458053                                      # Samantha's first secret prime
q = 3367900313                                      # Samantha's second secret prime
N = p*q                                             # Samantha's public modulus
print('N =', N)
e = 574812398758423897                              # Samantha's public verification exponent
print('e =', e)
d = extended_gcd(e, (p-1)*(q-1)//gcd(p-1, q-1))[1]  # Samantha's secret signing exponent, computed using the trapdoor information p and q

D = 1123581321345589                                # Document to be signed
print('D = ', D)
S = power(D, d, N)                                  # Signiture computed by Samantha using her secret exponent
print('S =', S)

print('Sent by Samantha?:', D == power(S, e, N))    # Verification conducted by Victor (or anyone else)

# Elgamal digital signitures

In [ ]:
# Find a large prime p and a primitive root a in F_p
p = generate_prime(6)
print('p =', p)
g = find_primitive(p)
print('g =', g)

# Samantha's chosen private exponent a, and her public key A
a = np.random.randint(2, p-1)
A = power(g, a, p)
print('A =', A)

# Document to be signed
D = np.random.randint(2, p)
print('D =', D)

# Using a random k prime to p-1 and her secret exponent a, Samantha creates a signiture (S1, S2)
k = np.random.randint(2, p)
while gcd(k, p-1) != 1:
  k = np.random.randint(2, p)
S1 = power(g, k, p)
S2 = ( (D - a*S1) * extended_gcd(k, p-1)[1] ) % (p-1)
print('(S1, S2) =', (S1, S2))

# Victor verifies the message is Samantha by checking that A^S1*S1^S2 = g^D mod p
print('From Samantha?:', ((power(A, S1, p) * power(S1, S2, p) * power(g, p-1-D, p)) % p) == 1)

# DSA

In [ ]:
# We will use the prime p from the previous computation.
# Recall that g1=7 was a primitive root modulo p

p = 1000231
g1 = 7

In [ ]:
# Note the following factorization of p-1.

(p-1) / (2*3*5*7*11*433)

In [ ]:
is_mrprime(433)

In [ ]:
# We will take q to be the largest prime dividing p-1
# g is an element of order q in F_p^\times

q=433
g = power(g1, (p-1)//q, p)

print('p =', p)
print('q =', q)
print('g =', g)

# Samantha chooses a secret signing exponent, and publishes A = g^a mod p
a = np.random.randint(2, q-1)
A = power(g, a, p)
print('A =', A)

# Document to be signed
D = np.random.randint(1, q-1)
print('D =', D)

# Samantha's signiture, using a random k
k = np.random.randint(2, q-1)
S1 = power(g, k, p) % q
S2 = ((D + a*S1) * power(k, q-2, q)) % q
print('(S1, S2) =', (S1, S2))

# Victor verifies:
V1 = (D * power(S2, q-2, q)) % q
V2 = (S1 * power(S2, q-2, q)) % q
print('Verified?:', S1 == ((power(g, V1, p)*power(A, V2, p))%p)%q )